##Importuri


In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

##Model


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


##Date Pb3

In [13]:
import json

data = [
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 1 first option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb1Sor(l):\n  txt = l.split()\n  lista = sorted(txt, reverse=True)\n  return lista[0]"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 1 second option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb1Sor2(l):\n  txt = l.split()\n  for i in range(len(txt)):\n        for j in range(0, len(txt)-i-1):\n            if txt[j] < txt[j+1]:\n                txt[j], txt[j+1] = txt[j+1], txt[j]\n  return txt[0]"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 2 first option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb2Euc1(x1,x2,y1,y2):\n  dx = int(x2)-int(x1)\n  dy = int(y2)-int(y1)\n  rez = math.sqrt(pow(dx,2) + pow(dy,2))\n  return rez"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 2 second option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb2Euc2(x1,x2,y1,y2):\n    dx = x2 - x1\n    dy = y2 - y1\n    distance = hypot(dx, dy)\n    return distance"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 3 first option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb3Cuv1(IP):\n  text = IP.split()\n  frecventa = {}\n  for cuv in text:\n      if cuv in frecventa:\n          frecventa[cuv] += 1\n      else:\n          frecventa[cuv] = 1\n  rezultat=[]\n  for key, value in frecventa.items():\n    if value == 1:\n      rezultat.append(key)\n  return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 3 second option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 4 first option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb4Rep1(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i != ' ':\n      if i in frecventa:\n          frecventa[i] += 1\n      else:\n          frecventa[i] = 1\n      if frecventa[i] == 2:\n        nr = i\n        break\n  return nr"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 4 second option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb4Rep2(l):\n  numere = set()\n  for i in l:\n    if i != ' ':\n      if i in numere:\n        return i\n      else:\n        numere.add(i)\n  return 0"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 5 first option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb5Max(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i in frecventa:\n        frecventa[i] += 1\n    else:\n        frecventa[i] = 1\n  lung = int(len(l) / 2)\n  rezultat = set()\n  for key, value in frecventa.items():\n    if value >= lung:\n      rezultat.add(key)\n  return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "I want problem 5 second option"
            },
            {
                "from": "assistant",
                "value": "Here it is:\ndef Pb5Max2(l):\n  lista = sorted(l)\n  lung = int(len(l) / 2)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for nr in lista:\n    aparitii[nr].append(nr)\n  for nr, ap in aparitii.items():\n    if len(ap) >= lung:\n      rezultat.append(ap[0])\n  return rezultat"
            }
        ]
    }
]



# Salvare în fișier JSON
with open('data.json', 'w') as f:
    json.dump(data, f, indent=2)


In [14]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# Obțineți șablonul de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"} # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Încărcarea datasetului
dataset = load_dataset('json', data_files='data.json', split='train')
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verificare a datelor preprocesate
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'text'],
    num_rows: 10
})


In [15]:
dataset[5]["conversations"]

[{'from': 'user', 'value': 'I want problem 3 second option'},
 {'from': 'assistant',
  'value': 'Here it is:\ndef Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat'}]

In [16]:
print(dataset[5]["text"])

<|begin_of_text|><|user|>
I want problem 3 second option<|end|>
<|assistant|>
Here it is:
def Pb3Cuv2(IP):
  text = IP.split()
  lista = sorted(text)
  rezultat=[]
  aparitii = defaultdict(list)
  for cuvant in lista:
    aparitii[cuvant].append(cuvant)
  rezultat = []
  for cuvant, lista_aparitii in aparitii.items():
    if len(lista_aparitii) == 1:
      rezultat.append(lista_aparitii[0])
  return rezultat<|end|>



##Structura output Pb3

In [17]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

##Setari train

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


##Train Pb3

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.153700
2,2.347800
3,2.014800
4,2.110600
5,1.604600
6,1.364800
7,1.174100
8,0.959400
9,0.728300
10,0.565600


In [21]:
model.save_pretrained("Ariana_Model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [33]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Ariana_Model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 30,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "I want problem 1 first option and nothing more"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 32, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
I want problem 1 first option and nothing more<|end|>
<|assistant|>
Here it is:
def Pb1Sor(l):
  txt = l.split()
  lista = sorted(txt, reverse=True)
  return lista[0]<


##Testare Pb3


In [36]:
messages = [
    {"from": "human", "value": "I want problem 1 second option and nothing more"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 80, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
I want problem 1 second option and nothing more<|end|>
<|assistant|>
Here it is:
def Pb1Sor2(l):
  txt = l.split()
  for i in range(len(txt)):
        for j in range(0, len(txt)-i-1):
            if txt[j] < txt[j+1]:
                txt[j], txt[j+1] = txt[j+1], txt[j]
  return txt[0]<|end|>



##Data PB2

In [37]:
import json

data = [
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm: \ndef Pb1Sor(l):\n  txt = l.split()\n  lista = sorted(txt, reverse=True)\n  return lista[0]"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is: \ndef Pb1Sor(l):\n    # Split the input string into a list of words\n    txt = l.split()\n    \n    # Sort the list of words in descending alphabetical order\n    lista = sorted(txt, reverse=True)\n\n        # Return the first word in the sorted list (which is the last word in alphabetical order)\n    return lista[0]\n"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb1Sor2(l):\n  txt = l.split()\n  for i in range(len(txt)):\n        for j in range(0, len(txt)-i-1):\n            if txt[j] < txt[j+1]:\n                txt[j], txt[j+1] = txt[j+1], txt[j]\n  return txt[0]"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb1Sor2(l):\n    # Split the input string into a list of words\n    txt = l.split()\n    \n    # Implement bubble sort to sort the words in descending alphabetical order\n    for i in range(len(txt)):\n        for j in range(0, len(txt)-i-1):\n            # Compare adjacent words and swap them if they are in the wrong order            if txt[j] < txt[j+1]:\n                txt[j], txt[j+1] = txt[j+1], txt[j]\n    \n    # Return the first word in the sorted list (which is the last word in alphabetical order)\n    return txt[0]"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm: \ndef Pb2Euc1(x1,x2,y1,y2):\n  dx = int(x2)-int(x1)\n  dy = int(y2)-int(y1)\n  rez = math.sqrt(pow(dx,2) + pow(dy,2))\n  return rez"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is: \ndef Pb2Euc1(x1, x2, y1, y2):\n    # Calculate the differences between the x and y coordinates\n    dx = int(x2) - int(x1)\n    dy = int(y2) - int(y1)\n    \n    # Calculate the Euclidean distance using the Pythagorean theorem\n    rez = math.sqrt(pow(dx, 2) + pow(dy, 2))\n    \n    # Return the calculated distance\n    return rez"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb2Euc2(x1,x2,y1,y2):\n    dx = x2 - x1\n    dy = y2 - y1\n    distance = hypot(dx, dy)\n    return distance"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is: \ndef Pb2Euc2(x1, x2, y1, y2):    # Calculate the differences between the x and y coordinates    dx = x2 - x1\n    dy = y2 - y1\n    \n    # Calculate the Euclidean distance using the hypot function from the math module\n    distance = hypot(dx, dy)\n    \n    # Return the calculated distance\n    return distance\n"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb3Cuv1(IP):\n  text = IP.split()\n  frecventa = {}\n  for cuv in text:\n      if cuv in frecventa:\n          frecventa[cuv] += 1\n      else:\n          frecventa[cuv] = 1\n  rezultat=[]\n  for key, value in frecventa.items():\n    if value == 1:\n      rezultat.append(key)\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb3Cuv1(IP):\n    # Split the input text into words (default separator is space)\n    text = IP.split()\n    \n    # Define a frequencydictionary to store the count of each word\n    frecventa = {}\n    \n    # Iterate through each word in the text\n    for cuv in text:\n        # Check if the word exists in the frequency dictionary\n        if cuv in frecventa:\n            # If it exists, increment its count\n            frecventa[cuv] += 1\n        else:\n            # Otherwise, add the new word to the dictionary with a count of 1\n            frecventa[cuv] = 1\n    \n    # Initialize an empty list to store words that appear only once\n    rezultat = []\n    \n    # Store words that appear only once in the result list\n    for key, value in frecventa.items():\n        if value == 1:\n            rezultat.append(key)\n    \n    # Return the list of words that appear only once\n    return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb3Cuv2(IP):\n    # Split the input text into words\n    text = IP.split()\n    \n    # Sort the list of words\n    lista = sorted(text)\n    \n    # Initialize an empty list to store unique words\n    rezultat = []\n    \n    # Initialize a defaultdict to store the occurrences of each word\n    aparitii = defaultdict(list)\n    \n    # Count the occurrences of each word\n    for cuvant in lista:\n        aparitii[cuvant].append(cuvant)\n\n    # Iterate through the dictionary items and append words with a single occurrence to the result list\n    for cuvant, lista_aparitii in aparitii.items():\n        if len(lista_aparitii) == 1:\n            rezultat.append(lista_aparitii[0])\n\n    # Return the list of unique words\n    return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb4Rep1(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i != ' ':\n      if i in frecventa:\n          frecventa[i] += 1\n      else:\n          frecventa[i] = 1\n      if frecventa[i] == 2:\n        nr = i\n        break\n  return nr"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb4Rep1(l):\n    # Initialize an empty dictionary to store the frequency of each character\n    frecventa = {}\n    \n    # Initialize a variable to store the repeating character\n    nr = 0\n    \n    # Iterate through each character in the input string\n    for i in l:\n        # Check if the character is not a space\n        if i != " ":\n            # Check if the character already exists in the frequency dictionary\n            if i in frecventa:\n                # Increment the frequency count of the character\n                frecventa[i] += 1\n            else:\n                # Add the character to the frequency dictionary with a count of 1\n                frecventa[i] = 1\n            \n            # Check if the frequency count of the character is equal to 2\n            if frecventa[i] == 2:\n                # If yes, store the character as the repeating character and exit the loop\n                nr = i\n                break\n    \n    # Return the repeating character\n    return nr"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb4Rep2(l):\n  numere = set()\n  for i in l:\n    if i != ' ':\n      if i in numere:\n        return i\n      else:\n        numere.add(i)\n  return 0"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb4Rep2(l):\n    # Initialize an empty set to store unique characters encountered in the input string\n    numere = set()\n    \n    # Iterate through each character in the input string\n    for i in l:\n        # Check if the character is not a space\n        if i != " ":\n            # Check if the character already exists in the set of unique characters\n            if i in numere:\n                # If yes, return the character as it's the first repeating character encountered\n                return i\n            else:\n                # If no, add the character to the set of unique characters\n                numere.add(i)\n    \n    # If no repeating character is found, return 0\n    return 0"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb5Max(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i in frecventa:\n        frecventa[i] += 1\n    else:\n        frecventa[i] = 1\n  lung = int(len(l) / 2)\n  rezultat = set()\n  for key, value in frecventa.items():\n    if value >= lung:\n      rezultat.add(key)\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb4Max(l):\n    # Initialize an empty dictionary to store the frequency of each character in the input string\n    frecventa = {}\n\n    # Iterate through each character in the input string\n    for i in l:\n        # Check if the character already exists in the frequency dictionary\n        if i in frecventa:\n            # If yes, increment its frequency count\n            frecventa[i] += 1\n        else:\n            # If no, initialize its frequency count to 1\n            frecventa[i] = 1\n    \n    # Calculate the minimum frequency threshold required for a character to be considered as a maximum\n    lung = int(len(l) / 2)\n    \n    # Initialize an empty set to store characters with frequencies greater than or equal to the threshold\n    rezultat = set()\n    \n    # Iterate through each key-value pair (character-frequency) in the frequency dictionary\n    for key, value in frecventa.items():\n        # Check if the frequency of the character is greater than or equal to the threshold\n        if value >= lung:\n            # If yes, add the character to the result set\n            rezultat.add(key)\n    \n    # Return the set containing characters with frequencies greater than or equal to the threshold\n    return rezultat"
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "Comment this alghorithm:\ndef Pb5Max2(l):\n  lista = sorted(l)\n  lung = int(len(l) / 2)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for nr in lista:\n    aparitii[nr].append(nr)\n  for nr, ap in aparitii.items():\n    if len(ap) >= lung:\n      rezultat.append(ap[0])\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Sure, here it is:\ndef Pb5Max2(l):\n    # Sort the input list of numbers\n    lista = sorted(l)\n    \n    # Calculate the minimum threshold required for a number to be considered as a maximum\n    lung = int(len(l) / 2)\n    \n    # Initialize an empty list to store numbers with frequencies greater than or equal to the threshold\n    rezultat = []\n    \n    # Initialize a defaultdict to store the occurrences of each number\n    aparitii = defaultdict(list)\n    \n    # Iterate through each number in the sorted list\n    for nr in lista:\n        # Append the number to its corresponding key in the defaultdict\n        aparitii[nr].append(nr)\n\n    # Iterate through each number and its list of occurrences in the defaultdict    for nr, ap in aparitii.items():\n        # Check if the number has occurrences greater than or equal to the threshold\n        if len(ap) >= lung:\n            # If yes, append the first occurrence of the number to the result list\n            rezultat.append(ap[0])\n\n    # Return the list containing numbers with occurrences greater than or equal to the threshold\n    return rezultat"
            }
        ]
    }
]



# Salvare în fișier JSON
with open('data1.json', 'w') as f:
    json.dump(data, f, indent=2)


In [41]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# Obțineți șablonul de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"} # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Încărcarea datasetului
dataset = load_dataset('json', data_files='data1.json', split='train')
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verificare a datelor preprocesate
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'text'],
    num_rows: 10
})


In [42]:
dataset[5]["conversations"]

[{'from': 'user',
  'value': 'Comment this alghorithm:\ndef Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat'},
 {'from': 'assistant',
  'value': 'Sure, here it is:\ndef Pb3Cuv2(IP):\n    # Split the input text into words\n    text = IP.split()\n    \n    # Sort the list of words\n    lista = sorted(text)\n    \n    # Initialize an empty list to store unique words\n    rezultat = []\n    \n    # Initialize a defaultdict to store the occurrences of each word\n    aparitii = defaultdict(list)\n    \n    # Count the occurrences of each word\n    for cuvant in lista:\n        aparitii[cuvant].append(cuvant)\n\n    # Iterate through the dictionary items and append words with a single occurrence to the result list\n 

In [43]:
print(dataset[5]["text"])

<|begin_of_text|><|user|>
Comment this alghorithm:
def Pb3Cuv2(IP):
  text = IP.split()
  lista = sorted(text)
  rezultat=[]
  aparitii = defaultdict(list)
  for cuvant in lista:
    aparitii[cuvant].append(cuvant)
  rezultat = []
  for cuvant, lista_aparitii in aparitii.items():
    if len(lista_aparitii) == 1:
      rezultat.append(lista_aparitii[0])
  return rezultat<|end|>
<|assistant|>
Sure, here it is:
def Pb3Cuv2(IP):
    # Split the input text into words
    text = IP.split()
    
    # Sort the list of words
    lista = sorted(text)
    
    # Initialize an empty list to store unique words
    rezultat = []
    
    # Initialize a defaultdict to store the occurrences of each word
    aparitii = defaultdict(list)
    
    # Count the occurrences of each word
    for cuvant in lista:
        aparitii[cuvant].append(cuvant)

    # Iterate through the dictionary items and append words with a single occurrence to the result list
    for cuvant, lista_aparitii in aparitii.items():
 

##Structura output Pb2

In [44]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

In [45]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


##Train Pb2

In [46]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.958000
2,1.037700
3,0.680100
4,0.621700
5,0.392700
6,0.278700
7,0.211000
8,0.224800
9,0.153900
10,0.138500


##Testare Pb2

In [52]:
messages = [
    {"from": "human", "value": "Comment this alghorithm:\ndef Pb1Sor2(l):\n  txt = l.split()\n  for i in range(len(txt)):\n        for j in range(0, len(txt)-i-1):\n            if txt[j] < txt[j+1]:\n                txt[j], txt[j+1] = txt[j+1], txt[j]\n  return txt[0]"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 150, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
Comment this alghorithm:
def Pb1Sor2(l):
  txt = l.split()
  for i in range(len(txt)):
        for j in range(0, len(txt)-i-1):
            if txt[j] < txt[j+1]:
                txt[j], txt[j+1] = txt[j+1], txt[j]
  return txt[0]<|end|>
<|assistant|>
Sure, here it is:
def Pb1Sor2(l):
    # Split the input string into a list of words
    txt = l.split()
    
    # Implement bubble sort to sort the words in descending alphabetical order
    for i in range(len(txt)):
        for j in range(0, len(txt)-i-1):
            # Compare adjacent words and swap them if they are in the wrong order            if txt[j] < txt[j+1]:
                txt[j], txt[j+1] = txt[j+1], txt[j]
    
    # Return the first word in the sorted list (which is the last word in alphabetical order)
    return txt[0]<|end|>
<|user|>



In [49]:
messages = [
    {"from": "human", "value": "Comment this alghorithm: \ndef Pb1Sor(l):\n  txt = l.split()\n  lista = sorted(txt, reverse=True)\n  return lista[0]"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 80, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
Comment this alghorithm: 
def Pb1Sor(l):
  txt = l.split()
  lista = sorted(txt, reverse=True)
  return lista[0]<|end|>
<|assistant|>
Sure, here it is: 
def Pb1Sor(l):
    # Split the input string into a list of words
    txt = l.split()
    
    # Sort the list of words in descending alphabetical order
    lista = sorted(txt, reverse=True)

        # Return the first word in the sorted list (which is the last word in alphabetical order)
    return lista[0]



##Data PB1


In [53]:
import json

data = [
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm? \n def Pb1Sor(l):\n  txt = l.split()\n  lista = sorted(txt, reverse=True)\n  return lista[0]"
            },
            {
                "from": "assistant",
                "value": "Sorts words in descending order and returns the first one."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm? \n def Pb1Sor2(l):\n  txt = l.split()\n  for i in range(len(txt)):\n        for j in range(0, len(txt)-i-1):\n            if txt[j] < txt[j+1]:\n                txt[j], txt[j+1] = txt[j+1], txt[j]\n  return txt[0]"
            },
            {
                "from": "assistant",
                "value": "Sorts words alphabetically using bubble sort and returns the first."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm? \n def Pb2Euc1(x1,x2,y1,y2):\n  dx = int(x2)-int(x1)\n  dy = int(y2)-int(y1)\n  rez = math.sqrt(pow(dx,2) + pow(dy,2))\n  return rez"
            },
            {
                "from": "assistant",
                "value": "This algorithm computes Euclidean distance."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb2Euc2(x1,x2,y1,y2):\n    dx = x2 - x1\n    dy = y2 - y1\n    distance = hypot(dx, dy)\n    return distance"
            },
            {
                "from": "assistant",
                "value": "Calculates Euclidean distance between two points in a plane."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb3Cuv1(IP):\n  text = IP.split()\n  frecventa = {}\n  for cuv in text:\n      if cuv in frecventa:\n          frecventa[cuv] += 1\n      else:\n          frecventa[cuv] = 1\n  rezultat=[]\n  for key, value in frecventa.items():\n    if value == 1:\n      rezultat.append(key)\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Counts unique words and returns those with frequency equals one."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Sorts words and returns unique ones with single occurrence."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb4Rep1(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i != ' ':\n      if i in frecventa:\n          frecventa[i] += 1\n      else:\n          frecventa[i] = 1\n      if frecventa[i] == 2:\n        nr = i\n        break\n  return nr"
            },
            {
                "from": "assistant",
                "value": "Returns the first repeated character excluding spaces in a string."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb4Rep2(l):\n  numere = set()\n  for i in l:\n    if i != ' ':\n      if i in numere:\n        return i\n      else:\n        numere.add(i)\n  return 0"
            },
            {
                "from": "assistant",
                "value": "Returns the first repeated non-space character in a string."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb5Max(l):\n  frecventa = {}\n  nr = 0\n  for i in l:\n    if i in frecventa:\n        frecventa[i] += 1\n    else:\n        frecventa[i] = 1\n  lung = int(len(l) / 2)\n  rezultat = set()\n  for key, value in frecventa.items():\n    if value >= lung:\n      rezultat.add(key)\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Returns the set of characters occurring more than half times."
            }
        ]
    },
    {
        "conversations": [
            {
                "from": "user",
                "value": "What does this algorithm?\n def Pb5Max2(l):\n  lista = sorted(l)\n  lung = int(len(l) / 2)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for nr in lista:\n    aparitii[nr].append(nr)\n  for nr, ap in aparitii.items():\n    if len(ap) >= lung:\n      rezultat.append(ap[0])\n  return rezultat"
            },
            {
                "from": "assistant",
                "value": "Returns the list of characters occurring more than half times."
            }
        ]
    }
]



# Salvare în fișier JSON
with open('data2.json', 'w') as f:
    json.dump(data, f, indent=2)

In [54]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# Obțineți șablonul de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"} # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Încărcarea datasetului
dataset = load_dataset('json', data_files='data2.json', split='train')
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verificare a datelor preprocesate
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'text'],
    num_rows: 10
})


In [55]:
dataset[5]["conversations"]

[{'from': 'user',
  'value': 'What does this algorithm?\n def Pb3Cuv2(IP):\n  text = IP.split()\n  lista = sorted(text)\n  rezultat=[]\n  aparitii = defaultdict(list)\n  for cuvant in lista:\n    aparitii[cuvant].append(cuvant)\n  rezultat = []\n  for cuvant, lista_aparitii in aparitii.items():\n    if len(lista_aparitii) == 1:\n      rezultat.append(lista_aparitii[0])\n  return rezultat'},
 {'from': 'assistant',
  'value': 'Sorts words and returns unique ones with single occurrence.'}]

In [56]:
print(dataset[5]["text"])

<|begin_of_text|><|user|>
What does this algorithm?
 def Pb3Cuv2(IP):
  text = IP.split()
  lista = sorted(text)
  rezultat=[]
  aparitii = defaultdict(list)
  for cuvant in lista:
    aparitii[cuvant].append(cuvant)
  rezultat = []
  for cuvant, lista_aparitii in aparitii.items():
    if len(lista_aparitii) == 1:
      rezultat.append(lista_aparitii[0])
  return rezultat<|end|>
<|assistant|>
Sorts words and returns unique ones with single occurrence.<|end|>



##Structura output Pb1

In [57]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

In [58]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


##Train Pb1

In [59]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.950400
2,1.046500
3,0.632600
4,0.406400
5,0.260900
6,0.154400
7,0.121400
8,0.113900
9,0.075400
10,0.051900


##Testare Pb1

In [61]:
messages = [
    {"from": "human", "value": "What does this algorithm? \n def Pb1Sor(l):\n  txt = l.split()\n  lista = sorted(txt, reverse=True)\n  return lista[0]"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 20, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
What does this algorithm? 
 def Pb1Sor(l):
  txt = l.split()
  lista = sorted(txt, reverse=True)
  return lista[0]<|end|>
<|assistant|>
Sorts words in descending order and returns the first one.<|end|>
<|user|


In [63]:
messages = [
    {"from": "human", "value": "What does this algorithm?\n def Pb3Cuv1(IP):\n  text = IP.split()\n  frecventa = {}\n  for cuv in text:\n      if cuv in frecventa:\n          frecventa[cuv] += 1\n      else:\n          frecventa[cuv] = 1\n  rezultat=[]\n  for key, value in frecventa.items():\n    if value == 1:\n      rezultat.append(key)\n  return rezultat"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 17, use_cache = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|user|>
What does this algorithm?
 def Pb3Cuv1(IP):
  text = IP.split()
  frecventa = {}
  for cuv in text:
      if cuv in frecventa:
          frecventa[cuv] += 1
      else:
          frecventa[cuv] = 1
  rezultat=[]
  for key, value in frecventa.items():
    if value == 1:
      rezultat.append(key)
  return rezultat<|end|>
<|assistant|>
Counts unique words and returns those with frequency equals one.<|end|>
<|
